In [1]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import load_model
import gym
from REINFORCE_helper import RunningVariance
import numpy as np
import multiprocessing

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
def run_episode(env, model, seed=42, explore=True):
    done = False
    if seed>0:
        env.seed(seed)
    obs = env.reset()
    rewards = []
    observations = []
    actions = []
    while not done:
        actions_dist = model.predict_proba(obs.reshape(-1, env.observation_space.shape[0]))
        if explore:
            sampled_action = np.random.choice(range(env.action_space.n), p=actions_dist[0])
        else:
            sampled_action = np.argmax(actions_dist[0])
        obs_, reward, done, _ = env.step(sampled_action)
        rewards.append(reward)
        observations.append(obs)
        actions.append(sampled_action)
        obs = obs_
    return observations, actions, rewards

In [ ]:
def run_it():
    print('procesando')
    env_cp = gym.make("CartPole-v1")
    from tensorflow.keras.models import load_model
    model = load_model('cartpole-trained.hdf5', compile=False)
    return run_episode(env_cp, model, seed=42, explore=True)

In [5]:
# run_it()

In [ ]:
from multiprocessing import Process
def runInParallel(*fns):
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        proc.append(p)
    for p in proc:
        p.join()
    return p

p = runInParallel(run_it, run_it, run_it)

procesando
procesando
procesando
Instructions for updating:
Please use `model.predict()` instead.

In [96]:
from multiprocessing import Pipe, Process

def child_process(func):
    """Makes the function run as a separate process."""
    def wrapper(*args, **kwargs):
        def worker(conn, func, args, kwargs):
            conn.send(func(*args, **kwargs))
            conn.close()
        parent_conn, child_conn = Pipe()
        p = Process(target=worker, args=(child_conn, func, args, kwargs))
        p.start()
        ret = parent_conn.recv()
        p.join()
        return ret
    return wrapper

@child_process
def keras_stuff():
    run_it()

In [97]:
keras_stuff()

procesando


KeyboardInterrupt: 

In [75]:
def monte_carlo(env, model, seed=42, N=500, explore=True, k=20):
  returns = []
  running_variance = RunningVariance()
  run_var = []
  run_mean = []
  for i in range(N):
    print(f'\rEpisodio: {i}', end='')
    observations, actions, rewards = run_episode(env, model, seed=seed, explore=explore)
    return_ = np.sum(rewards)
    running_variance.add(return_)
    run_var.append(running_variance.get_variance())
    run_mean.append(running_variance.get_mean())
    returns.append(return_)
  print()
  returns = np.array(returns)
  return run_mean, run_var, returns

In [19]:
env_cp = gym.make("CartPole-v1")

In [20]:
model = load_model('cartpole-trained.hdf5', compile=False)

In [26]:
%time run_mean_trained, run_var_trained, returns_trained = monte_carlo(env_cp, trained_model, seed=42, N=10, explore=False)

Episodio: 9
CPU times: user 36.9 s, sys: 785 ms, total: 37.7 s
Wall time: 36 s


In [29]:
run_mean_trained, run_var_trained, returns_trained

([277.0, 277.0, 277.0, 277.0, 277.0, 277.0, 277.0, 277.0, 277.0, 277.0],
 [1e-12, 1e-12, 1e-12, 1e-12, 1e-12, 1e-12, 1e-12, 1e-12, 1e-12, 1e-12],
 array([277., 277., 277., 277., 277., 277., 277., 277., 277., 277.]))

In [30]:
%time run_mean_trained_explore, run_var_trained_explore, returns_trained_explore = monte_carlo(env_cp, trained_model, seed=42, N=10, explore=True)

Episodio: 9
CPU times: user 40.7 s, sys: 766 ms, total: 41.5 s
Wall time: 39.6 s


In [32]:
run_mean_trained_explore, run_var_trained_explore, returns_trained_explore

([288.0,
  331.0,
  318.0,
  310.3333333333333,
  312.75,
  323.4,
  311.0,
  307.85714285714283,
  307.625,
  306.3333333333333],
 [1e-12,
  1e-12,
  337.999999999663,
  345.3333333331615,
  253.58333333324973,
  757.2999999998119,
  1528.399999999695,
  1342.8095238093006,
  1151.4107142855505,
  1022.499999999873],
 array([288., 331., 305., 295., 320., 366., 249., 289., 306., 296.]))